In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import os, sys, email
import gensim
from gensim.models import Doc2Vec
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from string import punctuation
import timeit
from sklearn.cluster import KMeans
from sklearn import metrics
import pylab as pl
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
start = timeit.default_timer()

In [2]:
data = pd.read_csv('processed_resumes_work_ADDED_JOB_TITLES.csv')

In [3]:
data.loc[data.converted_job_title.isnull()]['converted_job_title'] = 'Unknown'

/Users/kwheatley/anaconda/envs/python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [4]:
# Removed jobs without dates
data = data[~data.dates.isnull()]

# Removed jobs without description
data = data[~data.descript.isnull()]

In [5]:
converted_data = data

In [6]:
converted_data.columns 

Index(['Unnamed: 0', 'city', 'resume_id', 'container', 'role', 'company',
       'location', 'dates', 'descript', 'converted_job_title',
       'converted_experience_level'],
      dtype='object')

In [7]:
converted_data.columns = ['row_index', 'city', 'resume_id', 'container', 'role', 'company',
       'location', 'dates', 'descript', 'converted_job_title',
       'converted_experience_level']

In [8]:
converted_data.row_index = converted_data.row_index.astype(str)
converted_data.descript = converted_data.descript.str.strip()

In [9]:
converted_data['document'] = converted_data.converted_job_title.fillna("") + ' ' + \
                             converted_data.role.fillna("") + ' ' + \
                             converted_data.descript.fillna("")
converted_data['file'] = converted_data.resume_id + '-' + converted_data.row_index
# # corpus = list()

In [10]:
## This code in this and next cell is taken from the from Explore Enron notebook by Zichen Wang
## Helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [11]:
#Data cleaning
def email_cleaning(text):
    email = text.lower()    
    # clean and tokenize document string
    email_content = email.split()    
    word_list = []
    for i in email_content:
        x = 0
        if (('http' not in i) and ('@' not in i) and ('<.*?>' not in i) and i.isalnum() and (not i in stop_words)):
            word_list += [i]
        
    return word_list 

#Data Pre-processing
def preprocessing(text):    
    # remove numbers
    number_tokens = [re.sub(r'[\d]', ' ', i) for i in text]
    number_tokens = ' '.join(number_tokens).split()
     # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in number_tokens]
    # remove empty
    length_tokens = [i for i in stemmed_tokens if len(i) > 1]
    return length_tokens

In [12]:
LabeledSentence1 = gensim.models.doc2vec.TaggedDocument
all_content = []
texts = []
j=0
k=0
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
p_stemmer = PorterStemmer()
for em in converted_data.document:           
    #Data cleaning
    clean_content = email_cleaning(em)
    
    #Pre-processing
    processed_email = preprocessing(clean_content)
    
    # add tokens to list
    if processed_email:
        all_content.append(LabeledSentence1(processed_email,[j]))
        j+=1
        
    k+=1

print("Number of emails processed: ", k)
print("Number of non-empty emails vectors: ", j)

KeyboardInterrupt: 

In [ ]:
print(all_content[278])

In [ ]:
d2v_model = Doc2Vec(all_content, size = 2000, window = 10, min_count = 500, workers=7, dm = 1, 
                alpha=0.025, min_alpha=0.001)

In [ ]:
d2v_model.train(all_content, total_examples=d2v_model.corpus_count, epochs=10, start_alpha=0.002, end_alpha=-0.016)

In [ ]:
# shows the similar docs with id = 99
print (d2v_model.docvecs.most_similar(1))
#print(all_content[282])
#print(all_content[1])

In [ ]:
kmeans_model = KMeans(n_clusters=4, init='k-means++', max_iter=100)  
X = kmeans_model.fit(d2v_model.docvecs.doctag_syn0)
labels=kmeans_model.labels_.tolist()

In [ ]:
l = kmeans_model.fit_predict(d2v_model.docvecs.doctag_syn0)
pca = PCA(n_components=2).fit(d2v_model.docvecs.doctag_syn0)
datapoint = pca.transform(d2v_model.docvecs.doctag_syn0)

In [ ]:
plt.figure
label1 = ["#FFFF00", "#008000", "#0000FF", "#800080"]
color = [label1[i] for i in labels]
plt.scatter(datapoint[:, 0], datapoint[:, 1], c=color)

centroids = kmeans_model.cluster_centers_
centroidpoint = pca.transform(centroids)
plt.scatter(centroidpoint[:, 0], centroidpoint[:, 1], marker='^', s=150, c='#000000')
plt.show()

In [ ]:
stop = timeit.default_timer()
execution_time = stop - start

print(execution_time) #It returns time in sec

In [17]:
from gensim import corpora
import gensim
from gensim.models.doc2vec import TaggedDocument
import nltk
from nltk.corpus import stopwords
import csv
import numpy as np
import string
import re
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

In [27]:
def get_documents(r):
    i = 0
    for d in r:
        dt = get_words(d)
        doc = TaggedDocument([w for w in dt if w not in stopwords.words('english')], [i])
        i += 1
        yield doc
            
def get_words(r):
    r = re.sub('[^a-zA-Z ]', '', r.lower())
    words = nltk.word_tokenize(r)
    return [w for w in words if w not in stopwords.words('english')]

with open('processed_resumes_work_ADDED_JOB_TITLES.csv') as f:
    r = csv.reader(f)
#     i = 0
#     jobs = []
#     titles = []
#     docs = []
    for row in r:
        print(row)
        print(row[4] + row[5] + row[8])
#         jobs.append(row[7])
#         titles.append(row[3])
#         i += 1
#         if i >= 5000:
#             break

# sents_gen = get_documents(jobs)
# for s in sents_gen:
#     docs.append(s)
    
# titles = np.array(titles)

In [21]:
doc2vecModel =gensim.models.Doc2Vec(docs, size=20, iter=100)
vects = doc2vecModel.docvecs.doctag_syn0

/Users/kwheatley/anaconda/envs/python36/lib/python3.6/site-packages/gensim/models/doc2vec.py:355: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/Users/kwheatley/anaconda/envs/python36/lib/python3.6/site-packages/gensim/models/doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


AttributeError: 'str' object has no attribute 'words'